### Check data correctness

In [1]:
import pyodbc
import pandas as pd
from pathlib import Path
from datetime import date
import numpy as np
import win32com.client as win32

# find the past month
start_date = date.today() - pd.DateOffset(weeks=1)
start_date = start_date.strftime('%Y-%m-%d')
print(start_date)
# start_date = '2024-06-05'
today = date.today().strftime('%Y-%m-%d')
# today = '2024-03-22'

2024-12-03


In [2]:
# connection
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

# download shortge from SQL
cursor.execute(f"SELECT * FROM OPS.GPS_tbl_ops_shortage_ext where ReportDate >= '{start_date}'")
shortage_ext = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
shortage_ext['HP_PN'] = shortage_ext['HP_PN'].str.replace('\n', ',', regex=False)
shortage_ext['FV'] = shortage_ext['FV'].str.replace('\n', ',', regex=False)

# download PNFV from SQL
cursor.execute("SELECT * FROM OPS.GPS_tbl_ops_PN_FV")
PNFV = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
PNFV = PNFV.rename({'Descr':'FV'})

#### Check last FD date is empty

In [3]:
# create the last FD null row
lastFDnull = shortage_ext[shortage_ext['last FD date'].isnull()].copy()

if len(lastFDnull)>0:
    lastFDnull['Issue'] = 'Last FD is blank'
    lastFDnull['last FD date'] = 'Blank'
    print('Last FD is blank:', len(lastFDnull))
else:
    lastFDnull = pd.DataFrame()

Last FD is blank: 16


#### Check P1~P3 sum <> total shortage

In [4]:
# P1 + P2 + P3 != Total
QtyMismatch = []
for i, row in shortage_ext.iterrows():
    P123 = row['P1'] + row['Net P2'] + row['Net P3']
    total = row['Total Shortage Qty'] 
    if P123 != total:
        QtyMismatch.append(row)
    else:
        pass

if len(QtyMismatch) > 0:
    df_QtyMismatch = pd.DataFrame(QtyMismatch)
    df_QtyMismatch['Issue'] = 'P1~P3 sum <> total shortage'
else:
    df_QtyMismatch = pd.DataFrame()

print(f'P1~P3 sum != total shortage: {len(QtyMismatch)}')

P1~P3 sum != total shortage: 0


#### Check BT > P1

In [5]:
# BT qty > P1
BT_P1 = []
for i, row in shortage_ext.iterrows():
    BT = row['BT shortage']
    P1 = row['P1'] 
    if BT > P1:
        BT_P1.append(row)
    else:
        pass

if len(BT_P1) > 0:
    df_BT_P1 = pd.DataFrame(BT_P1)
    df_BT_P1['Issue'] = 'BT shortage > P1'
else:
    df_BT_P1 = pd.DataFrame()

print('BT shortage > P1:', len(BT_P1))

BT shortage > P1: 0


#### Check BT > total shortage

In [6]:
# BT > total shortage
BT_total = []
for i, row in shortage_ext.iterrows():
    BT = row['BT shortage']
    total = row['Total Shortage Qty'] 
    if BT > total:
        BT_total.append(row)
    else:
        pass

if len(BT_total) > 0:
    df_BT_total = pd.DataFrame(BT_total)
    df_BT_total['Issue'] = 'BT shortage > Total shortage'
else:
    df_BT_total = pd.DataFrame()

print('BT > total:', len(BT_total))

BT > total: 0


#### Check working on upside > total shortage

In [7]:
# working on upside > total shortage
upside_total = []
for i, row in shortage_ext.iterrows():
    upside = row['Working on upside']
    total = row['Total Shortage Qty'] 
    if upside > total:
        upside_total.append(row)
    else:
        pass

if len(upside_total) > 0:
    df_upside_total = pd.DataFrame(upside_total)
    df_upside_total['Issue'] = 'Working on upside > Total shortage'
else:
    df_upside_total = pd.DataFrame()

print('upside > total:', len(upside_total))

upside > total: 0


#### Check PN not in PNFV table

In [8]:
# PN not in PNFV table
PN_list = PNFV.PN.tolist()


df_PN_notExist = []
for index, row in shortage_ext.iterrows():
    hp_pns = row['HP_PN'].split(',')
    if all(hp_pn.strip() in PN_list for hp_pn in hp_pns):
        pass
    else:
        row['Issue'] = 'HP_PNs do not exist in the PNFV table'
        df_PN_notExist.append(row)

if len(df_PN_notExist) > 0:
    df_PN_notExist = pd.DataFrame(df_PN_notExist)
else:
    df_PN_notExist = pd.DataFrame()

print('PN not in PNFV:', len(df_PN_notExist))

PN not in PNFV: 0


#### Check FV not in PNFV table

In [9]:
# FV not in PNFV table
FV_list = [fv.strip() for fv in PNFV.Descr.tolist()]
 

df_FV_notExist = []
for index, row in shortage_ext.iterrows():
    fvs = row['FV'].split(',')
    if all(fv.strip() in FV_list for fv in fvs):
        pass
    else:
        row['Issue'] = 'FVs do not exist in the PNFV table'
        df_FV_notExist.append(row)

if len(df_FV_notExist) > 0:
    df_FV_notExist = pd.DataFrame(df_FV_notExist)
else:
    df_FV_notExist = pd.DataFrame()

print('FV not in PNFV:', len(df_FV_notExist))

FV not in PNFV: 0


In [10]:
df_FV_notExist

""


In [11]:
shortage_test = shortage_ext.copy()

shortage_test = shortage_test[(shortage_test['BuyerName'] == 'WangWinson(CW)') & (shortage_test['ReportDate'] == '2024-06-12')]

shortage_test

,ODM,Item,Commodity,FV,Platform,P1,Net P2,Net P3,Total Shortage Qty,BT shortage,Working on upside,ReportDate,last FD date,HP_PN,BuyerName


#### Concat all result

In [12]:
# data type management
try:
    error_all = pd.concat([lastFDnull, df_QtyMismatch, df_BT_P1, df_BT_total, df_upside_total, df_FV_notExist, df_PN_notExist], axis=0)
    error_all['P1'] = error_all['P1'].astype(int)
    error_all['Net P2'] = error_all['Net P2'].astype(int)
    error_all['Net P3'] = error_all['Net P3'].astype(int)
    error_all['BT shortage'] = error_all['BT shortage'].astype(int)
    error_all['Working on upside'] = error_all['Working on upside'].astype(int)
    error_all['Total Shortage Qty'] = error_all['Total Shortage Qty'].astype(int)
except:
    error_all = pd.DataFrame()
    print('No error found')

In [13]:
# concat same errors
if len(error_all) > 0:
    # fist group by all column except issue, then agg function will aggreate the words
    concatenated_issues = error_all.groupby(list(error_all.columns.difference(['Issue']))).agg({'Issue': ', '.join}).reset_index()
    error_all = concatenated_issues.drop_duplicates()
    error_all = error_all[['BuyerName','Issue','ODM','Item','Commodity','FV','Platform','P1','Net P2','Net P3','Total Shortage Qty','BT shortage','Working on upside','ReportDate','last FD date','HP_PN',]]
else:
    pass

# Send email

In [14]:
if (len(error_all) > 0):
    # sent to list
    To_list = 'gps.taiwan.nb.buy-sell@hp.com'
    CC_list = 'spencer.cheng1@hp.com; howie.chang1@hp.com'

    # start creating the email
    mail = win32.Dispatch("Outlook.Application").CreateItem(0)
    mail.To = To_list
    mail.CC = CC_list
    mail.Subject = 'BSP B/S shortage dashboard alert ' + '<From Report Date: '+ start_date+' to ' + today + '>'

    # Step 1: Highlight only the BuyerName values
    error_all['BuyerName'] = error_all['BuyerName'].apply(lambda name: f'<span style="background-color: yellow;">{name}</span>')

    # Step 2: Convert the DataFrame to HTML, allowing HTML content within cells
    html_error_all = error_all.to_html(escape=False, index=False)

    # message --> use the html content from the previous step 1&2 to 
    signature = "<br><br>Best Regards,<br>Newcomen Project Team"
    message = f"""Hi team,<br><br>
    Here are items that need your amendment in shortage dashboard. (data starting from: <font color='blue'>{start_date})</font><br>
    Please complete it by next workday and contact project team if any question, thank you.<br>
    * Due to database refresh timing gap, items you have done action might still be on the list here. Please ignore if you have done action. Let project team know if your amendment is not completed for days.<br><br>
    {html_error_all}<br>{signature}"""

    # make sure email fully initialized, and only write after <body>
    mail.GetInspector 
    index = mail.HTMLbody.find('>', mail.HTMLbody.find('<body')) 

    # generate email
    mail.HTMLBody = message
    mail.display()

else:
    To_list = 'gps.taiwan.nb.buy-sell@hp.com'
    CC_list = 'spencer.cheng1@hp.com; howie.chang1@hp.com'

    # start creating the email
    mail = win32.Dispatch("Outlook.Application").CreateItem(0)
    mail.To = To_list
    mail.CC = CC_list
    mail.Subject = 'BSP B/S shortage dashboard alert ' + '<From Report Date: '+ start_date+' to ' + today + '>'
    signature = "<br><br>Best Regards,<br>Newcomen Project Team"
    message = f"""Hi team,<br><br>
    No item needed for amendment. (data starting from: <font color='blue'>{start_date})</font><br>
    Thank you.<br><br>"""

    # make sure email fully initialized, and only write after <body>
    mail.GetInspector 
    index = mail.HTMLbody.find('>', mail.HTMLbody.find('<body')) 

    # generate email
    mail.HTMLBody = message
    mail.display()
    print('No error found, no email sent')